In [1]:
import glob
import pandas as pd
import numpy as np
import cv2
from skimage import feature
import pickle

In [2]:
# Load Data
data = pd.read_csv('x_y2.csv')


In [3]:
# Split into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(data)[9000:],data[['y0','y1','y2','y3','y4','y5']][9000:], test_size=.2, random_state=42)

In [4]:
batch_size = 100
x_pixels = int(100)
y_pixels = int(100)
n_classes = 6

In [5]:
def get_data(data_x,data_y,i):
    i = i*batch_size
    x_data_cnn = []
    x_data_neural = []
       
    type_class_check = type(np.array([1,2,3]))
    for counter in range(i,i+batch_size):
        try:
            #Insert Path to file where all images are present
            image = cv2.imread('/home/pratik/Desktop/be_project/images/Data/Images/working/'+data_x.iloc[counter]['name']+'.jpg',cv2.IMREAD_GRAYSCALE)
            if type(image) == type_class_check:
                x_data_cnn.append(cv2.resize(image,(x_pixels,y_pixels)))
                
            else:
                #print('nai hai')
                x_data_cnn.append(np.zeros([x_pixels,y_pixels]))
            x_data_neural.append(data_x.iloc[counter][['age','sex','s0','s1','s2','s3','s4','s5','s5']])
        except ValueError:
            print("Missed data point")
            
    y_data = data_y[['y0','y1','y2','y3','y4','y5']][i:i+batch_size]
    #x_data = np.zeros([batch_size,x_pixels,y_pixels])
    return x_data_cnn,x_data_neural, y_data


In [6]:
import tensorflow as tf

n_classes = 6
batch_size = 200

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

#The place holder to hold the values of new x and y at different times.
x_neural = tf.placeholder('float', [batch_size, 9])
x_cnn = tf.placeholder('float', [batch_size, x_pixels*y_pixels])
y = tf.placeholder('float',[batch_size,n_classes])

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([9, n_nodes_hl1], name='weights1')),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]),name='biases1')}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2]), name='weights2'),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]), name='biases2')}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3]),name='weights3'),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]), name='biases3')}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes]),name='output_nn'),
                    'biases':tf.Variable(tf.random_normal([n_classes]),name='biases_output')}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    #output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']
    return l3


def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32]),name='W_conv1'),
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64]),name='W_conv2'),
               'W_fc':tf.Variable(tf.random_normal([(int(x_pixels//4))*(int(y_pixels//4))*64,1024]),name='W_fc'),
               'out':tf.Variable(tf.random_normal([1024, n_classes]),name='outc')}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32]), name='b_conv1'),
               'b_conv2':tf.Variable(tf.random_normal([64]), name='b_conv2'),
               'b_fc':tf.Variable(tf.random_normal([1024]), name='b_fc'),
               'out':tf.Variable(tf.random_normal([n_classes]), name='outb')}

    x = tf.reshape(x, shape=[-1, x_pixels, y_pixels, 1])

    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)

    fc = tf.reshape(conv2,[-1, (int(x_pixels//4))*(int(y_pixels//4))*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)
    #print(fc.shape)
    #output = tf.matmul(fc, weights['out'])+biases['out']
    #print(output.shape)
    return fc






def train_neural_network(x_neural,x_cnn):
    output_combined = {'weights':tf.Variable(tf.random_normal([1524, n_classes]),name='finw'),
                    'biases':tf.Variable(tf.random_normal([n_classes]),name='finb')}
    output_cnn = convolutional_neural_network(x_cnn)
    output_neural = neural_network_model(x_neural)
    print(output_cnn.shape,output_neural.shape)
    output = tf.concat([output_cnn,output_neural],1)
    print(output.shape)
    prediction = tf.add(tf.matmul(output,output_combined['weights']), output_combined['biases'])
    #prediction = tf.nn.relu(raw_pred)
    #prediction = neural_network_model(x_neural)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = prediction,labels = y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    trial = tf.argmax(prediction,1)
    correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))
    
    hm_epochs = 5
    
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        
        sess.run(tf.initialize_all_variables())
        

        
        import datetime

        tf.summary.scalar('Loss', cost)
        tf.summary.scalar('Accuracy', accuracy)
        merged = tf.summary.merge_all()
        logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
        writer = tf.summary.FileWriter(logdir, sess.graph)
        for epoch in range(hm_epochs):
            epoch_loss = 0
            epoch_accuracy = 0
            for i in range(len(X_train)//batch_size):
                print((i*20000)/len(X_train))
                epoch_x_cnn,epoch_x_neural, epoch_y = get_data(X_train,y_train,i)
                epoch_x_cnn = np.array(epoch_x_cnn).reshape(batch_size,x_pixels*y_pixels)
                _, c, a, t = sess.run([optimizer, cost,accuracy,trial], feed_dict={x_cnn: epoch_x_cnn, x_neural: epoch_x_neural, y: epoch_y})
                #o = sess.run([cost], feed_dict={x_cnn: epoch_x_cnn, x_neural: epoch_x_neural, y: epoch_y})
                #print(f.shape, o.shape)
                epoch_loss += c
                epoch_accuracy += a
                print("current accuracy: "+str(a)+"current loss: "+str(c),t)
                summary = sess.run(merged, {x_cnn: epoch_x_cnn, x_neural: epoch_x_neural, y: epoch_y})
                writer.add_summary(summary, i)
                
                
        
                
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            save_path = saver.save(sess, "/home/pratik/combinationfins6.ckpt")
            print("Model saved in path: %s" % save_path)
        
        test_x_cnn, test_x_neural, test_y = get_data(X_test,y_test,0)
        test_x_cnn = np.array(test_x_cnn).reshape(batch_size,x_pixels*y_pixels)
        print('Accuracy:',sess.run([accuracy,trial],feed_dict={x_cnn: test_x_cnn, x_neural: test_x_neural, y: test_y}))
        
train_neural_network(x_neural,x_cnn)



/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


(200, 1024) (200, 500)
(200, 1524)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.0
current accuracy: 0.11current loss: 998955.2 [2 4 2 4 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 4 4 2 2 2 2
 2 2 2 4 4 2 2 2 2 2 2 2 4 2 5 2 2 2 4 2 2 2 2 2 2 2 4 0 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 4 2 2 4 2 4 4 4 2 2 2 2 2 2 2
 2 4 2 2 2 2 2 2 2 2 2 2 4 2 2 2 4 2 2 2 2 4 2 0 2 2 2 2 4 4 2 0 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 5 2 2 5 2 2 4 2 2 2 4 2 2 4 2 2 2 2 2 2 2 2 2
 4 2 4 2 2 2 2 2 2 2 2 2 2 2 2]
2.0165355918531964
current accuracy: 0.23current loss: 562224.8 [0 4 2 2 4 0 5 5 4 2 2 1 2 2 2 0 1 0 4 4 0 2 2 2 4 4 2 4 0 0 2 2 4 2 2 4 4
 4 4 0 5 4 0 2 0 4 4 4 2 1 0 5 4 2 2 2 4 4 2 4 2 4 4 2 0 2 2 4 2 0 2 0 4 2
 2 4 0 2 0 2 4 0 4 5 2 0 0 0 4 4

KeyboardInterrupt: 

In [ ]:
data['y0'].sum()

In [ ]:
X_train